In [21]:
import os, json
import pandas as pd
import re
from pythainlp.util import thai_strptime
from datetime import datetime

with open("data/provinces.json", 'r', encoding='utf-8') as file:
    LOCS = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python
# map
def map_function(item):    
    return  item["province"]

# อาจได้ผลลัพธ์ที่มี None
PROVINCE_LIST = list(map(map_function, LOCS))

def location_detection(text):       
    # ใช้ regex เพื่อค้นหาชื่อจังหวัด
    province_pattern = r"จังหวัด\s*([^\s|]*)|จ\.\s*([^\s|]*)|เมือง\s*([^\s|]*)"
    matches = re.findall(province_pattern, text)

    # ดึงชื่อจังหวัดออกมา
    provinces = [match[0] or match[1] for match in matches]
    unique_provinces = set(provinces)  # ลบชื่อจังหวัดที่ซ้ำกัน

    # แสดงผล
    if unique_provinces:
        # print(f"เหตุการณ์เกิดขึ้นที่จังหวัด: {', '.join(unique_provinces)}")
        clean_unique_provinces = []
        for p in PROVINCE_LIST:
            if p in " | ".join(list(unique_provinces)):
                clean_unique_provinces.append(p)
        return clean_unique_provinces if len(clean_unique_provinces) > 0 else []
    else:
        # print("ไม่พบชื่อจังหวัดในข้อความ")
        return []

# ฟังก์ชันแปลงวันที่ภาษาไทยเป็นรูปแบบ datetime
def thai_date_to_standard(thai_date_str):
    # แปลงวันที่ภาษาไทยเป็น datetime object
    # 12 ก.ย. 2567 10:59 น.
    date_object = thai_strptime(thai_date_str, "%d %B %Y %H:%M น.")

    # แปลง datetime object เป็น ISO 8601 (YYYY-MM-DD)
    return date_object.strftime("%Y-%m-%d")



directory = 'data-detail'
data = []
for filename in os.listdir(directory):
    if ".json" not in filename:
        continue

    # LOAD NEWS
    with open(f"{directory}/{filename}", 'r', encoding='utf-8') as file:
        news = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python
    
    # LOCATIONS
    locations = []
    for entity in news["ner_obj"]["ner"]["entities"]:
        if entity["type"] == "LOCATION" and entity["value"].strip() != "": 
            locations.append(entity["value"].strip())
    location_string = " | ".join(locations)
    province = " | ".join(location_detection(location_string))

    # PUBLISHED DATE
    # ตัวอย่างวันที่ภาษาไทย
    standard_date = thai_date_to_standard(news["publish_date"])

    item = {
        "id" : news["tags"][-1],
        "title" : news["title"],
        "url" : news["url"],
        # "location_string" : location_string,
        "province" : province,
        # "publish_date" : news["publish_date"],
        "ISO8601" : standard_date,
    }
    data.append(item)

df = pd.DataFrame(data)
filename = "data/analysis.xlsx"
df.to_excel(filename, index=False)  # บันทึกเป็น Excel
print(f"Export File: {filename}")


# แปลง DataFrame เป็น JSON
json_file = "data/analysis.json"
df.to_json(json_file, orient="records", lines=False, force_ascii=False, indent=4)

Export File: data/analysis.xlsx
